In [1]:
import pymongo
from pymongo import MongoClient
import pprint

In [2]:
#连接mongodb
client = MongoClient()
print(client)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [3]:
def handler_doc():
    """操作文档"""
    
    #创建数据库及集合
    py_collection = client.python.py_collection
    
    #插入单个文档
    #py_collection.insert({"_id":1,"name":"张三","age":18})
    #py_collection.insert_one({"_id":2,"name":"李四","age":25})
    
    #插入多个文档
    doc1 = {"_id":3,"name":"王五","age":20}
    doc2 = {"_id":4,"name":"赵六","age":21}
    doc3 = {"_id":5,"name":"孙七","age":22}
    doc4 = {"_id":6,"name":"周八","age":23}
    #py_collection.insert_many([doc1,doc2,doc3,doc4])
    
    #更改数据
    #filter:更改条件，update：更改的参数，upsert：
    doc5 = py_collection.update_one(filter={'name':'李四'},update={'$set':{'name':'吴九','age':19}})#更改单个文档
    print(doc5)
    doc6 = py_collection.update_many(filter={'name':'王五'},update={'$set':{'name':'郑十','age':29}})#更改多个文档
    print(doc6)
    
    #update_doc = {'name':'吴九','age':19}
    #py_collection.update_one(filter={'name':'李四'},update={'$set':update_doc})
    
    #删除数据
    doc7 = py_collection.delete_one({'name':"周八"})#删除单条
    print(doc7)
    doc8 = py_collection.delete_many({'name':"张三"})#删除单条
    print(doc8)

In [ ]:
handler_doc()

In [ ]:
#查询集合中的文档
collection_1 = client.python.py_collection
for col in collection_1.find():
    pprint.pprint(col)

In [ ]:
#自定义文档输出函数
def print_doc(x):
    for i in x:
        print(i)

In [ ]:
print_doc(collection_1.find())

In [ ]:
#and操作
def test_and():
    doc = client.python.py_collection.find({
        "$and":[
            {"_id":{"$gte":3,"$lte":4}},
            {"age":{"$gte":4}}
        ]
    })
    print_doc(doc)

In [ ]:
test_and()

In [ ]:
#or操作
def test_or():
    doc = client.python.py_collection.find({
        "$or":[
            {"_id":{"$gte":0,"$lte":1}},
            {"age":{"$lte":4}},
            {"name":"孙七"}
        ]
    })
    print_doc(doc)

In [ ]:
test_or()

In [ ]:
#正则表达式
def test_regex():
    #doc = client.python.py_collection.find({'name':{"$regex":'^吴'}})
    doc = client.python.py_collection.find({'name':{"$regex":'^吴.*?(九|十)$'}})#以“吴”开头、以“九”或者“十”结尾
    print_doc(doc)

In [ ]:
test_regex()

In [ ]:
#数组
doc1 = {"name":"王五","age":20,"score":[88,90,86,95]}
doc2 = {"name":"赵六","age":21,"score":[90,96,87,90]}
doc3 = {"name":"孙七","age":22,"score":[78,92,85,88]}
doc4 = {"name":"周八","age":23,"score":[66,77,88,99]}
client.python.py_collection.insert_many([doc1,doc2,doc3,doc4])

def test_array():
    doc = client.python.py_collection.find(
    {},
    {
        "_id":0,
        "name":1,
        "age":1,
        "score":{"$slice":[0,2]},
    })
    print_doc(doc)

In [ ]:
test_array()

In [ ]:
#排序
def test_sort():
    #按姓名正序
    doc = client.python.py_collection.find({},{"name":1}).sort("name",pymongo.ASCENDING)
    print("#按姓名正序：")
    print_doc(doc)
    
    #按年龄倒叙，按_id正序
    doc = client.python.py_collection.find({},{"age":-1,"_id":1}).sort([("age",pymongo.DESCENDING),("_id",pymongo.ASCENDING)])
    print("\n#按年龄倒叙，按_id正序：")
    print_doc(doc)

In [ ]:
test_sort()

In [ ]:
#分页
def test_page(pageSize,pageNum):
    #doc = client.python.py_collection.find().skip(0).limit(2)
    doc = client.python.py_collection.find().skip((pageNum-1)*pageSize).limit(pageSize)
    print_doc(doc)

In [ ]:
test_page(2,2)

In [ ]:
#分组聚合
db = client.Student_Task

def test_group():
    doc = db.Task2_MXQ.aggregate([{"$group":{"_id":"$顾客编号","消费总额":{"$sum":"$销售金额"}}},
                        {"$match":{"消费总额":{"$gte":100}}}])
    print_doc(doc)

In [ ]:
test_group()

In [ ]:
#排序限制过滤
def test_sort_limit_skip():
    doc = db.Task2_MXQ.aggregate([
        {"$group":{"_id":"$顾客编号","消费总额":{"$sum":"$销售金额"}}},
        {"$sort":{"消费总额":-1}},
        #{"$skip":10},
        {"$limit":2}  
    ])
    print_doc(doc)

In [ ]:
test_sort_limit_skip()

In [ ]:
#随机获取
def test_sample():
    doc = db.Task2_MXQ.aggregate([
        {"$sample":{"size":3}}
    ])
    print_doc(doc)

In [ ]:
test_sample()

In [ ]:
#操作字符串
def test_str():
    
    #截取字符串
    doc = db.Task2_MXQ.aggregate([
        {
            "$project":{
                "顾客编号":1,
                "大类名称":1,
                "截取编码":{"$substr":["$商品编码",0,2]}
            }
        }
    ])
    print_doc(doc)
    
    #拼接
    doc = db.Task2_MXQ.aggregate([
        {
            "$project":{
                "顾客编号":1,
                "大类名称":1,
                "中类_小类名称":{"$concat":["$中类名称","_","$小类名称"]}
            }
        }
    ])
    print_doc(doc)
    
    #英文大小写转换($toLower/$toUpper)
    doc = db.Task2_MXQ.aggregate([
        {
            "$project":{
                "顾客编号":1,
                "大类名称":1,
                "商品编码":{"$toLower":"$商品编码"}
            }
        }
    ])
    print_doc(doc)

In [ ]:
test_str()